In [1]:
import dash
import pandas as pd
import plotly.express as px
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

In [2]:
app = dash.Dash(__name__)

df = pd.read_csv('cc_data.csv')
df[:10]

,Date,Date_Time,Num_Coins,Reported_Hash,Calculated_Hash,Diff
0,0,0,0.000000,0,0,0.000000
1,2021-11-21,13:33:51.397040,595835.737839,30,29,0.000000
2,2021-11-21,13:34:53.458088,595835.737839,30,29,0.000000
3,2021-11-21,14:01:40.669087,597117.691131,30,19,-350.001428
4,2021-11-21,14:03:51.583825,597259.004981,30,19,-141.313850
5,2021-11-21,14:04:12.533754,597259.004981,30,24,0.000000
6,2021-11-21,14:04:37.615924,597406.838645,30,24,-147.833664
7,2021-11-21,14:11:19.639080,597406.838645,30,19,0.000000
8,2021-11-21,17:31:46.397327,600176.355207,30,0,2769.516563
9,2021-11-21,17:33:12.344673,600488.756394,30,29,312.401187


In [3]:
d = ['2021-12-08','2021-12-07','2021-12-06']
dft = df[df['Date'].isin(d)]

In [4]:
dft

,Date,Date_Time,Num_Coins,Reported_Hash,Calculated_Hash,Diff
427,2021-12-06,08:40:43.807021,853655.826878,31,0,575.341569
428,2021-12-06,08:55:54.254383,853738.036102,31,10,82.209224
429,2021-12-06,09:11:04.723468,854149.373997,31,44,411.337894
430,2021-12-06,09:26:15.206623,854457.766753,31,53,308.392757
431,2021-12-06,09:41:25.698916,854614.970016,31,53,157.203263
...,...,...,...,...,...,...
534,2021-12-08,17:02:52.087989,916559.280202,31,29,488.569428
535,2021-12-08,17:18:02.423410,916851.864046,31,19,292.583844
536,2021-12-08,17:33:12.669599,917562.709331,31,24,710.845285
537,2021-12-08,17:48:23.079726,918015.084026,31,34,452.374694


#### Our data is not in the best format to plot the way we want. Well have to do a slight transformation using pd.wide_to_long

In [5]:
l = [0,1,3,4]
dfM = dft.iloc[:,l]
dfM = pd.melt(dfM, id_vars=['Date','Date_Time'],var_name='hash', value_name='rate')
dfM['Date'].unique()

array(['2021-12-06', '2021-12-07', '2021-12-08'], dtype=object)

In [6]:
fig = px.bar(dfM,x='Date_Time', color='hash', y='rate', title='Coin Tracker: Reported vs Calculated Hash', barmode='group')

In [7]:
app.layout = html.Div(children=[
    html.H1(children='My First Dash for Coin'),
    
    html.Div(children='Hash Rate by the hour'),
    dcc.Graph(
        id='coin-graph',
        figure=fig
    )
    
])


In [ ]:
if __name__ =='__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Dec/2021 22:16:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 22:16:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 22:16:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 22:16:48] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 22:16:48] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


In [19]:
#guess you app.callback had to be written with you function
@app.callback(
    Output('bar_coin', 'figure'),
    Input('day_slider', 'value'))
def update_figure(selected_day):
    filter_df = dfM[dfM.Date == selected_day]
    
    fig = px.bar(dfM,x='Date_Time', color='hash', y='rate', title='Coin Tracker: Reported vs Calculated Hash', barmode='group')
    
    fig.update_layout(transition_duration=500)
    
    return fig

app.layout = html.Div(children=[
    html.H1(children='MyFirst Dash For Coin'),
    
    html.Div(children='''
          Hash Rate by the Hour
          '''),
    
    dcc.Graph(id='bar_coin'),
       dcc.Slider(
        id='day_slider',
        min=dfM['Date'].min(),
        max=dfM['Date'].max(),
        value=dfM['Date'].min(),
        marks = {str(date): str(date) for date in dfM['Date'].unique()}
    )
])